In [1]:
import os
import warnings
import random
import shutil
from multiprocessing import cpu_count

import numpy as np
import tifffile as tiff
from PIL import Image
import cv2
import matplotlib.pyplot as plt
import seaborn as sns
from psutil import virtual_memory
from torch.cuda import is_available, device_count, get_device_name, get_arch_list

warnings.filterwarnings("ignore")
random.seed(241)

gpu_num = '5'
os.environ["CUDA_VISIBLE_DEVICES"] = gpu_num # Set to GPU 0 on Training
os.environ["WORLD_SIZE"] = "1"

### Check Detected Device by Torch

In [2]:
import warnings
warnings.filterwarnings('ignore')

# Check PyTorch version and available CUDA architectures
print('PyTorch Compute Capability')
print(get_arch_list())

# Check if CUDA is available
if is_available():
    # Get the number of available GPUs
    num_gpus = device_count()
    print(f"Number of available GPUs: {num_gpus}")
    
    # Print information about each GPU
    for i in range(num_gpus):
        gpu_name = get_device_name(i)
        print(f"GPU {i}: {gpu_name}")
else:
    print("CUDA is not available. GPU support is not enabled.")

# Get system RAM size
ram_size = virtual_memory().total / (1024 ** 3)  # Convert bytes to gigabytes
print(f"Total RAM size: {ram_size:.2f} GB")

# Get CPU name
cpu_name = None
with open("/proc/cpuinfo", "r") as f:
    for line in f:
        if "model name" in line:
            cpu_name = line.strip().split(": ")[1]
            break
print(f"CPU name: {cpu_name}")

# Get number of CPU threads
num_cores = cpu_count() // 2
print(f"Number of CPU threads: {num_cores}")


PyTorch Compute Capability
['sm_50', 'sm_60', 'sm_70', 'sm_75', 'sm_80', 'sm_86', 'sm_90']
Number of available GPUs: 1
GPU 0: NVIDIA H100 80GB HBM3
Total RAM size: 2015.36 GB
CPU name: Intel(R) Xeon(R) Platinum 8480+
Number of CPU threads: 112


### Define Path and Parameter

In [3]:
model = "yolov5n-seg.yaml" # Architecture Recepie
data_name = "temp" # Dataset Name
fold = 5 #CV Fold

train_image_path = 'dataset/train/s2_image/' # Train image path
train_label_path = 'dataset/train/mask/' # Train label path

evaluation_image_path = 'dataset/evaluation/s2_image/' # Evaluation image path
evaluation_label_path = 'dataset/evaluation/mask/' # Evaluation label path

wd = os.getcwd() # Get the current directory path

### Preprocess Image

In [4]:
def get_file_names(path, remove_extension=False) :
    '''
    This function is to get all the file names in a folder.
    remove_extension argument is for removing the file extension

    remove_extension=False
        [file_1.jpg, file_2.tiff, file_3.txt]
    remove_extension=True
        [file_1, file_2, file_3]

    Filtering of .ipynb_checkpoints, .gitignore, and __pycache__
    '''
    
    if remove_extension :
        return sorted([i.split('.')[0] for i in os.listdir(path) if i[0] != '.' or i[0] != '_'])
    else :
        return sorted([i for i in os.listdir(path) if i[0] != '.'])

In [5]:
def read_images(path) :
    '''
    Read all the .tif images inside a folder and 
    return a python list that consist of numpy array of the image.
    '''

    # Get all the file names from the given path
    file_names = get_file_names(path)

    # Read all the images in the path and store it in a python list
    images = []
    for file_name in file_names :
        file_path = os.path.join(path, file_name)
        # Save as float32
        image = tiff.imread(file_path).astype(np.float32)

        images.append(image)
    
    return images

In [6]:
def select_band(band, images) :
    '''
    Select the individual band of each image from a list of images
    extracted_channels = [[channel_n of image_1], [channel_n of image_2], ...]

    the extracted_channels is a python list consisting of 2D (h, w) numpy array
    representing the single channel from each image.
    '''

    extracted_channel = []
    for image in images :
        # Select a single band from h, w, c format
        extracted_channel.append(image[:, :, band])

    return extracted_channel

In [7]:
def norm_min_max(arr) :
    '''
    Normalized all the images inside the python list from the global minimum
    and maximum.

    The normalized image is multiplied by 255 so the range is [0, 255]
    '''
    # Min and max value used
    min_val, max_val = 0, 10000
    
    normalized_img = []
    for img in arr :
        img[img > 10000] = 10000 # Clip value above 10000
        
        img = ( (img - min_val) / (max_val - min_val) ) * 255
        normalized_img.append(img)

    return normalized_img

def norm_min_max_image(image, min_val=None, max_val=None) :
    '''
    Normalized single image and multiplied by 255
    '''

    # Get the min max value from the image if the min and max value is not given
    if min_val == None and max_val == None :
        min_val, max_val = np.min(image), np.max(image)
        
    image = ( (image - min_val) / (max_val - min_val) ) * 255

    return image

In [8]:
def formula(bandA, bandB) :
    '''
    Formula for finding ratio between two spectral bands
    '''
    new_band = []
    for i in range(len(bandA)) :
        a, b = bandA[i], bandB[i]
        processed_band = (a-b) / ((a+b) + 1e-10)

        # The ratio will have a range of [-1, 1]
        processed_band = norm_min_max_image(processed_band, min_val=-1, max_val=1)
        
        new_band.append(processed_band)

    return new_band

In [9]:
def export_images(images, source_path, export_path) :
    '''
    Export the preprocessed image to .tif format

    images : array of the stacked images (h, w, c)
    source_path : the source image path to get the original file name
    export_path : the path for the .tif image to be written
    '''
    # Get the original file names
    file_names = get_file_names(source_path)

    for image, file_name in zip(images, file_names) :
        H, W, C = image.shape

        # Resize the image to match the inputsize as close as possible
        image = cv2.resize(image, (W * 10, H * 10), interpolation=cv2.INTER_LINEAR)
        
        file_path = os.path.join(export_path, file_name)
        tiff.imwrite(file_path, image) # Write the preprocessed image

In [10]:
def export_labels(label_source_path, label_export_path) :
    '''
    Convert the binary .tif mask to YOLO label .txt format

    label_source_path : the folder path containing .tif binary mask
    label_export_path : the folder for the .txt labels to be written
    '''
    
    file_names = get_file_names(label_source_path)

    for file_name in file_names :
        file_path = os.path.join(label_source_path, file_name)
        mask = tiff.imread(file_path)
        H, W = mask.shape

        # Resize the image 10 times with linear interpolation to match the image size
        # !!! Please match the size and interpolation of the image for optimal mask representation
        mask = cv2.resize(mask, (W * 10, H * 10), interpolation=cv2.INTER_LINEAR)
        _, mask = cv2.threshold(mask, 0, 255, cv2.THRESH_BINARY) # Convert to 0 and 255 mask

        H, W = mask.shape
        contours, hierarchy = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
        # Convert the contours to polygons
        polygons = []
        for cnt in contours:
            if cv2.contourArea(cnt) > 0:
                polygon = []
                for point in cnt:
                    x, y = point[0]
                    polygon.append(x / W)
                    polygon.append(y / H)
                polygons.append(polygon)
    
        # Write the polygons
        file_name = file_name.replace('mask', 's2_image')[:-4]+'.txt'
        with open(os.path.join(label_export_path, file_name), 'w') as f:
            for polygon in polygons:
                for p_, p in enumerate(polygon):
                    if p_ == len(polygon) - 1:
                        f.write('{}\n'.format(p))
                    elif p_ == 0:
                        f.write('0 {} '.format(p))
                    else:
                        f.write('{} '.format(p))
    
            f.close()

In [11]:
def make_data_directory(data_name) :
    '''
    Create a folder and subfolder structure for the preprocessed images and
    converted labels

    |-train_yolov5.ipynb
    |
    |
    |-preprocessed_data
    |    |-{data_name}
    |    |    |-train
    |    |    |    |-images
    |    |    |    |    |-preprocessed_image.tif
    |    |    |    |    |- ...
    |    |    |    |    
    |    |    |    |-labels
    |    |    |         |-preprocessed_image.txt
    |    |    |         |- ...
    |    |    |-evaluation
    |    |    |    |-images
    |    |    |    |    |-preprocessed_image.tif
    |    |    |    |    |- ...
    |    |    |    |    
    |    |    |    |-labels
    |    |    |         |-preprocessed_image.txt
    |    |    |         |- ...
    |    |
    |    |-{other_data_name}
    |    |    | ...
    |    
    '''

    # Define the main directory
    working_dir = os.getcwd()
    container_dir = os.path.join(working_dir, 'preprocessed_data')
    data_dir = os.path.join(container_dir, data_name)

    # Directory for images and labels on each set
    sub_dirs = [
        'train/images',
        'train/labels',
        'evaluation/images',
        'evaluation/labels'
    ]

    for sub_dir in sub_dirs :
        os.makedirs(os.path.join(data_dir, sub_dir), exist_ok=True)

In [12]:
def preprocess_images(image_source_path, image_export_path) :
    # Output the source path, export path, and the number of images
    print('Source Path : ', image_source_path)
    print('Export Path : ', image_export_path)
    print('Images Count : ', len(get_file_names(image_source_path)))
    
    # Array of train data images
    image_source = read_images(image_source_path)
    
    # Num of channels to be extracted from the original .tif file
    ch_num = [0,1,2,3,4,5,6,7,8,9,10,11]
    channels = []
    normalized_channels = []
    
    # Select the first 12 channels from the image source
    for i in ch_num :
        ch = select_band(i, image_source)
        normalized_ch = norm_min_max(ch)
        
        channels.append(ch)
        normalized_channels.append(normalized_ch)
    
    # Create a new channel from permutation of channels (2,3,4)
    for i in range(1, 4) :
        for j in range(1, 4) :
            if i != j :
                ch = formula(normalized_channels[i], normalized_channels[j])
                normalized_channels.append(ch)
    
    # Create a new channel from permutation of channels (10,11)
    for i in range(10, 12) :
        for j in range(10, 12) :
            if i != j :
                ch = formula(normalized_channels[i], normalized_channels[j])
                normalized_channels.append(ch)
    
    # Create the image to h, w, c format
    images = [np.stack([normalized_channels[i][j] for i in range(len(normalized_channels))], axis=-1, dtype=np.float32) for j in range(len(normalized_channels[0]))]

    return images

In [13]:
# Create the folder structure based on the path
make_data_directory(data_name)

# Define the train source path and the export path from user defined path
train_image_source_path = os.path.join(wd, train_image_path)
train_image_export_path = os.path.join(wd, 'preprocessed_data', data_name, 'train', 'images')
train_label_source_path = os.path.join(wd, train_label_path)
train_label_export_path = os.path.join(wd, 'preprocessed_data', data_name, 'train', 'labels')

# Define the evaluation source path and the export path from user defined path
evaluation_image_source_path = os.path.join(wd, evaluation_image_path)
evaluation_image_export_path = os.path.join(wd, 'preprocessed_data', data_name, 'evaluation', 'images')
evaluation_label_source_path = os.path.join(wd, evaluation_label_path)
evaluation_label_export_path = os.path.join(wd, 'preprocessed_data', data_name, 'evaluation', 'labels')

In [14]:
# Preprocess train images
train_images = preprocess_images(
    train_image_source_path,
    train_image_export_path,
)
 
print('Exporting Train Data...')
# Export the preprocessed train images
export_images(train_images, train_image_source_path, train_image_export_path)
# Export the label to yolo format
export_labels(train_label_source_path, train_label_export_path)

print('Train Data Exported')

Source Path :  /datadisk2/c241_ml02/workspace/dataset/train/s2_image/
Export Path :  /datadisk2/c241_ml02/workspace/preprocessed_data/temp/train/images
Images Count :  2066
Exporting Train Data...
Train Data Exported


In [15]:
# Preprocess evaluation images
evaluation_images = preprocess_images(
    evaluation_image_source_path,
    evaluation_image_export_path,
)

print('Exporting Evaluation Data...')
# Export the preprocessed evaluation images
export_images(evaluation_images, evaluation_image_source_path, evaluation_image_export_path)
# Write the dummy label
export_labels(evaluation_label_source_path, evaluation_label_export_path)

print('Evaluation Data Exported')

Source Path :  /datadisk2/c241_ml02/workspace/dataset/evaluation/s2_image/
Export Path :  /datadisk2/c241_ml02/workspace/preprocessed_data/temp/evaluation/images
Images Count :  2066
Exporting Evaluation Data...
Evaluation Data Exported


### Create 5 Fold Cross Validation

In [16]:
def create_fold(file_names, fold) :
    # Shuffle the images
    random.shuffle(file_names)

    # Define each fold size
    fold_sizes = [len(file_names) // fold for i in range(fold)]

    # Distribute the remaining divider
    for i in range(len(file_names) - sum(fold_sizes)) :
        fold_sizes[-1] += 1

    # Assign each images to the fold
    fold_data = []
    for i in range(len(fold_sizes)) :
        data = file_names[sum(fold_sizes[:i]): sum(fold_sizes[:i+1])]
        fold_data.append(data)

    return fold_data

In [17]:
def make_train_directory(data_name, n_fold) :
    '''
    Create a folder and subfolder structure for
    the each fold of preprocessed image

    |-train_yolov5.ipynb
    |
    |
    |-train_data
    |    |-{data_name}
    |    |    |-configs
    
    |    |    |    |-evaluation_config.yaml
    |    |    |    |-fold_1.yaml
    |    |    |    |-...
    |    |    |    
    |    |    |-evaluation
    |    |    |    |-images
    |    |    |    |-labels
    |    |    |
    |    |    |-fold_1
    |    |    |    |-train
    |    |    |    |    |-images
    |    |    |    |    |-labels
    |    |    |    |
    |    |    |    |-val
    |    |    |         |-images
    |    |    |         |-labels
    |    |    |    
    |    |    |-fold_n
    |    |    |    |-...
    |    |    
    |    |-{other_data_name}
    |    |    | ...
    |    
    '''

    # Define the path for each foler
    working_dir = os.getcwd()
    container_path = os.path.join(working_dir, 'train_data')
    data_path = os.path.join(container_path, data_name)
    config_path = os.path.join(data_path, 'configs')
    evaluation_path = os.path.join(data_path, 'evaluation')

    evaluation_images_path = os.path.join(evaluation_path, 'images')
    evaluation_labels_path = os.path.join(evaluation_path, 'labels')
    

    # Create the folders
    os.makedirs(container_path, exist_ok=True)
    os.makedirs(data_path, exist_ok=True)
    os.makedirs(config_path, exist_ok=True)
    os.makedirs(evaluation_path, exist_ok=True)

    os.makedirs(evaluation_images_path, exist_ok=True)
    os.makedirs(evaluation_labels_path, exist_ok=True)

    # Make a subfolder for each folds
    fold_paths = [f'fold_{(i+1)}' for i in range(n_fold)]
    
    for n in fold_paths :
        fold_path = os.path.join(data_path, n)
        train_path = os.path.join(fold_path, 'train')
        val_path = os.path.join(fold_path, 'val')
        
        os.makedirs(fold_path, exist_ok=True)
        os.makedirs(train_path, exist_ok=True)
        os.makedirs(val_path, exist_ok=True)

        for path in [train_path, val_path] :  
            images_dir = os.path.join(path, 'images')
            labels_dir = os.path.join(path, 'labels')

            os.makedirs(images_dir, exist_ok=True)
            os.makedirs(labels_dir, exist_ok=True)

In [18]:
def copy_data(data_name, source_path, destination_path, part) :
    '''
    Function for copy the preprocessed image and its labels
    '''
    for file_name in data_name :
        image_file_name = file_name + '.tif'
        label_file_name = file_name + '.txt'

        # Path for images
        image_source_path = os.path.join(source_path, 'images', image_file_name)
        image_destination_path = os.path.join(destination_path, part, 'images', image_file_name)

        # Path for labels
        label_source_path = os.path.join(source_path, 'labels', label_file_name)
        label_destination_path = os.path.join(destination_path, part, 'labels', label_file_name)

        shutil.copy(image_source_path, image_destination_path) # Copy the image
        shutil.copy(label_source_path, label_destination_path) # Copy the labels

In [19]:
def make_fold(data_name, fold_data, source_path, destination_path) :
    '''
    Make a fold with 1 fold as validation data and 
    the rest as train data.
    '''
    
    for i in range(len(fold_data)) :
        # Path for fold_n
        fold_destination_path = os.path.join(destination_path, f'fold_{(i+1)}')

        # Find the fold for the validation data
        train_data = []
        for fold in fold_data :
            if fold_data[i] == fold :
                copy_data(fold_data[i], source_path, fold_destination_path, 'val')
            else :
                # Add the fold data to train data
                train_data += fold

        # Copy the rest fold to the train path
        copy_data(train_data, source_path, fold_destination_path, 'train')
            

In [20]:
def write_configs(fold, data_path, config_path) :    
    '''
    Write the config for the train and validation data for each fold
    '''
    for i in range(fold) :
        
        fold_path = os.path.join(data_path, f'fold_{(i+1)}') # The main fold folder

        # Write the config as yaml
        with open(os.path.join(config_path, f'fold_{(i+1)}.yaml'), 'w') as yaml :
            config = f"""path: {fold_path}
train: train
val: val

nc: 1
names: ['solarpanel']"""

            yaml.write(config)

def write_evaluation_config(data_path, config_path) :
    '''
    Write the config of the evaluation data with the train and val data is
    theh evaluation data itself
    '''
    config_path = os.path.join(config_path, 'evaluation.yaml')

    with open(config_path, 'w') as yaml :
        config = f"""path: {data_path}
train: evaluation
val: evaluation

nc: 1
names: ['solarpanel']"""

        yaml.write(config)

In [21]:
def check_image_count(destination_path, fold) :
    '''
    Function to check the file number of each fold on the train and validation
    '''

    paths = [f'fold_{(i)}' for i in range(1, fold+1)]
    for path in paths :
        fold_path = os.path.join(destination_path, path)
        train_image_path = os.path.join(fold_path, 'train', 'images')
        train_label_path = os.path.join(fold_path, 'train', 'labels')
        val_image_path = os.path.join(fold_path, 'val', 'images')
        val_label_path = os.path.join(fold_path, 'val', 'labels')
        
        print(f'=== {path} ===')
        print('Train Images:', len(os.listdir(train_image_path)))
        print('Train Labels :', len(os.listdir(train_label_path)))
        print('Val Images :', len(os.listdir(val_image_path)))
        print('Val Labels :', len(os.listdir(val_label_path)))

    fold_path = os.path.join(destination_path, 'evaluation')
    evaluation_image_path = os.path.join(fold_path, 'images')
    evaluation_label_path = os.path.join(fold_path, 'labels')

    print('=== evaluation ===')
    print('Evaluation Images:', len(os.listdir(evaluation_image_path)))
    print('Evaluation Labels:', len(os.listdir(evaluation_label_path)))

In [22]:
# Define the path for preprocessed data and train data
working_dir = os.getcwd()
source_path = os.path.join(working_dir, 'preprocessed_data', data_name)
destination_path = os.path.join(working_dir, 'train_data', data_name)
config_path = os.path.join(destination_path, 'configs')
train_source_path = os.path.join(source_path, 'train')
evaluation_source_path = os.path.join(source_path, 'evaluation')


make_train_directory(data_name, fold) # Create the folders

# Get the file names
file_names = get_file_names(os.path.join(source_path, 'train', 'images'), remove_extension=True)
# Create the fold data based on the file names
fold_data = create_fold(file_names, fold)
# Create the fold and copy the data
make_fold(data_name, fold_data, train_source_path, destination_path)
# Write config for each fold
write_configs(fold, destination_path, config_path)

# Copy preprocessed evaluation data
evaluation_name = get_file_names(os.path.join(evaluation_source_path, 'images'), remove_extension=True)
copy_data(evaluation_name, evaluation_source_path, destination_path, 'evaluation')
# Write config for evaluation
write_evaluation_config(destination_path, config_path)

In [23]:
# Check the file numbers
print('\n=== Image Count per Fold ===\n')
check_image_count(destination_path, fold)


=== Image Count per Fold ===

=== fold_1 ===
Train Images: 1653
Train Labels : 1653
Val Images : 413
Val Labels : 413
=== fold_2 ===
Train Images: 1653
Train Labels : 1653
Val Images : 413
Val Labels : 413
=== fold_3 ===
Train Images: 1653
Train Labels : 1653
Val Images : 413
Val Labels : 413
=== fold_4 ===
Train Images: 1653
Train Labels : 1653
Val Images : 413
Val Labels : 413
=== fold_5 ===
Train Images: 1652
Train Labels : 1652
Val Images : 414
Val Labels : 414
=== evaluation ===
Evaluation Images: 2066
Evaluation Labels: 2066


### Train YOLOv5

In [24]:
# Config for the model size, defined in the begining of the code
model_config_path = os.path.join(wd, 'yolov5', 'models', 'segment', model)
# Config for the train data (fold_1 of the data name is the default)
data_path = os.path.join(wd, 'train_data', data_name, 'configs', 'fold_1.yaml')

In [25]:
# Train the yolov5
!python yolov5/segment/train.py \
    --img 256 \
    --batch 128 \
    --epochs 10 \
    --data {data_path} \
    --cfg {model_config_path} \
    --device {gpu_num} \
    --no-overlap \
    --optimizer AdamW \
    --patience 300 \
    --name {data_name}


# Weight of the yolov5 will be saved at
# yolov5/runs/train-seg/{data_name}/weights

segment/train: weights=yolov5/yolov5s-seg.pt, cfg=/datadisk2/c241_ml02/workspace/yolov5/models/segment/yolov5n-seg.yaml, data=/datadisk2/c241_ml02/workspace/train_data/temp/configs/fold_1.yaml, hyp=yolov5/data/hyps/hyp.scratch-low.yaml, epochs=10, batch_size=128, imgsz=256, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, bucket=, cache=None, image_weights=False, device=5, multi_scale=False, single_cls=False, optimizer=AdamW, sync_bn=False, workers=8, project=yolov5/runs/train-seg, name=temp, exist_ok=False, quad=False, cos_lr=False, label_smoothing=0.0, patience=300, freeze=[0], save_period=-1, seed=0, local_rank=-1, mask_ratio=4, no_overlap=True
github: ⚠️ YOLOv5 is out of date by 18 commits. Use 'git pull' or 'git clone https://github.com/ultralytics/yolov5' to update.
YOLOv5 🚀 v7.0-313-g712de55a Python-3.12.3 torch-2.3.1+cu121 CUDA:5 (NVIDIA H100 80GB HBM3, 81117MiB)

hyperparameters: lr0=0.0005, lrf=0.0005, momentum=0.937, weight

### Evaluation Mask Extraction

In [26]:
def create_evaluation_folder(wd, data_name) :
    '''
    Create a folder for storing the binary mask of the evaluation set

    |-train_yolov5.ipynb
    |
    |-masks
         |-{data_name}
              |-evaluation_mask_0.tif
              |-...
    '''
    mask_folder = os.path.join(wd, 'masks', data_name)
    os.makedirs(mask_folder, exist_ok=True)

In [27]:
def delete_validation_path(wd, data_name) :
    '''
    Delete the internal evaluation data inside the val-seg on the run folder
    of yolov5.
    '''
    evaluation_result_path = os.path.join(wd, 'yolov5', 'runs', 'val-seg', data_name)
    shutil.rmtree(evaluation_result_path) # Delete the whole folder

In [29]:
# Define the paths
evaluate_path = os.path.join(wd, 'yolov5', 'segment', 'evaluate.py') # Evaluation to print the mask
model_path = os.path.join(wd, 'yolov5', 'runs', 'train-seg', data_name, 'weights', 'best.pt') # Model to be evaluated
data_path = os.path.join(wd, 'train_data' , data_name, 'configs', 'evaluation.yaml') # Config to the evaluation data

# Create the folder to store the mask
create_evaluation_folder(wd, data_name)

# Evaluate
!python {evaluate_path} \
    --weights {model_path} \
    --data {data_path} \
    --conf-thres {conf} \
    --imgsz 256 \
    --name {data_name}

# Delete the internal evaluation data
delete_validation_path(wd, data_name)

Traceback (most recent call last):
  File "/datadisk2/c241_ml02/workspace/yolov5/segment/evaluate.py", line 576, in <module>
    opt = parse_opt()
          ^^^^^^^^^^^
  File "/datadisk2/c241_ml02/workspace/yolov5/segment/evaluate.py", line 532, in parse_opt
    opt.data = check_yaml(opt.data)  # check YAML
               ^^^^^^^^^^^^^^^^^^^^
  File "/datadisk2/c241_ml02/workspace/yolov5/utils/general.py", line 476, in check_yaml
    return check_file(file, suffix)
           ^^^^^^^^^^^^^^^^^^^^^^^^
  File "/datadisk2/c241_ml02/workspace/yolov5/utils/general.py", line 504, in check_file
    assert len(files), f"File not found: {file}"  # assert file was found
           ^^^^^^^^^^
AssertionError: File not found: /datadisk2/c241_ml02/workspace/train_data/restructured_l/configs/evaluation.yaml


FileNotFoundError: [Errno 2] No such file or directory: '/datadisk2/c241_ml02/workspace/yolov5/runs/val-seg/restructured_l'

In [ ]:
# Define paths for the dummy masks and written masks
mask_folder = os.path.join(wd, 'masks', data_name)
extracted_masks = get_file_names(mask_folder)
evaluation_file_names = get_file_names(evaluation_label_path)

# Loop for write empty mask
for file_name in evaluation_file_names :
    if file_name not in extracted_masks :
        h, w = tiff.imread(os.path.join(evaluation_label_path, file_name)).shape
        empty_mask = np.zeros((h, w), dtype=np.uint8)

        output_mask_path = os.path.join(mask_folder, file_name)
        tiff.imwrite(output_mask_path, empty_mask)

In [ ]:
!zip -r masks_l_{conf}.zip masks/restructured_l